In [1]:
import requests
from io import StringIO
import pandas as pd


res = \
requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
print(res)
df = pd.read_table(StringIO(res.text), sep='\t')
df.head(5)

<Response [200]>


,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [2]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [3]:
sentences = []

In [4]:
for url in urls:
    res = requests.get(url)
    df = pd.read_csv(StringIO(res.text), header=None, sep='\t', on_bad_lines='skip')
    sentences.extend(df[1].tolist())
    sentences.extend(df[2].tolist())
sentences = list(set(sentences))
len(sentences)

14507

In [5]:
sentences = [s for s in sentences if type(s) == str]
len(sentences)

14506

In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_emb = model.encode(sentences)
sentence_emb.shape

(14506, 768)

In [11]:
import faiss
d = sentence_emb.shape[1]
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [12]:
index.add(sentence_emb)
print(index.ntotal)

14506


In [20]:
xq = model.encode(['python programming lang'])
xq.shape

(1, 768)

In [21]:
%%time
k = 4
D, I = index.search(xq, k)

CPU times: user 9.29 ms, sys: 0 ns, total: 9.29 ms
Wall time: 11.6 ms


In [22]:
print(I)

[[12933  3763  5885 13468]]


In [28]:
for i in I[0]:
  print(sentences[i])

create code, write a computer program.
write a computer program.
create code or computer programs
This integrates with Rational PurifyPlus and allows developers to work in supported versions of Java, Visual C# and Visual Basic .NET.


In [35]:
import numpy as np
rvecs = []
for i in I[0]:
  rvecs.append(index.reconstruct(int(i)))
rvecs = np.stack(rvecs)
rvecs.shape

(4, 768)

In [38]:
quantizer = faiss.IndexFlatL2(d)
nlist = 50
index = faiss.IndexIVFFlat(quantizer, d, nlist)
index.is_trained

False

In [39]:
%%time
index.train(sentence_emb)
index.is_trained

CPU times: user 365 ms, sys: 24 ms, total: 389 ms
Wall time: 389 ms


True

In [40]:
%%time
index.add(sentence_emb)

CPU times: user 62.1 ms, sys: 7.04 ms, total: 69.1 ms
Wall time: 68.5 ms


In [42]:
index.ntotal

14506

In [50]:
xq = model.encode(["Someone sprints with a football"])
xq.shape

(1, 768)

In [51]:
%%time
D, I = index.search(xq, k)
I

CPU times: user 56 µs, sys: 963 µs, total: 1.02 ms
Wall time: 710 µs


array([[ 3875, 13085,  4137,   894]])

In [52]:
for i in I[0]:
  print(sentences[i])

Two groups of people are playing football
A person playing football is running past an official carrying a football
A football player kicks the ball.
A football player is running past an official carrying a football


In [53]:
index.nprobe = 10

In [54]:
%%time
D, I = index.search(xq, k)
I

CPU times: user 2.77 ms, sys: 0 ns, total: 2.77 ms
Wall time: 1.54 ms


array([[ 1358, 13716,  3875, 13085]])

In [55]:
for i in I[0]:
  print(sentences[i])

A group of football players is running in the field
A group of people playing football is running in the field
Two groups of people are playing football
A person playing football is running past an official carrying a football


In [56]:
m = 8
bits = 8
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)
index.is_trained

False

In [57]:
index.train(sentence_emb)
index.is_trained

True

In [58]:
xq = model.encode(['fielder running to catch a crickext ball'])
xq.shape

(1, 768)

In [59]:
%%time
D, I = index.search(xq, 5)
I

CPU times: user 288 µs, sys: 0 ns, total: 288 µs
Wall time: 297 µs


array([[-1, -1, -1, -1, -1]])